# SANS2D: Q1D workflow for a single file

In [ ]:
import scipp as sc
from ess import loki, sans
import scippneutron as scn

## Loading data files

In [ ]:
ds = sc.Dataset()

#Using only one-forth of the full spectra 245760 (reserved for first detector)
spectrum_size =  245760//4

# Make common tof bins so that dataset items are aligned
tof_bins = sc.linspace(dim='tof', start=0, stop=100000, num=2, unit='us')

# Wavelength binning
wavelength_bins = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=141, unit='angstrom')

# Sample measurement
ds['sample'] = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063114.nxs'),
                                   spectrum_size=spectrum_size, tof_bins=tof_bins)
# Direct measurement is with the empty sample holder/cuvette
ds['direct'] = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063091.nxs'),
                                   spectrum_size=spectrum_size, tof_bins=tof_bins)

In [ ]:
ds

## Apply offsets to pixel positions

In [ ]:
# Custom SANS2D position offsets
sample_pos_z_offset = 0.053 * sc.units.m
bench_pos_y_offset = 0.001 * sc.units.m
# There is some uncertainity here
monitor4_pos_z_offset = -6.719 * sc.units.m

# Geometry transformation
x_offset = -0.09288 * sc.units.m
y_offset = 0.08195 * sc.units.m
# z_offset = 0.0 * sc.units.m

In [ ]:
ds.coords["pixel_width"] = 0.0035 * sc.units.m
ds.coords["pixel_height"] = 0.002033984375 * sc.units.m

# Change sample position
ds.coords["sample_position"].fields.z += sample_pos_z_offset
# Apply bench offset to pixel positions
ds.coords["position"].fields.y += bench_pos_y_offset

for key in ds:
    ds[key].attrs["monitor4"].value.coords["position"].fields.z += monitor4_pos_z_offset

# Now shift pixels positions to get the correct beam center
ds.coords['position'].fields.x += x_offset
ds.coords['position'].fields.y += y_offset

## Mask bad pixels

In [ ]:
mask_edges = (
    (sc.abs(ds.coords['position'].fields.x - x_offset) > sc.scalar(0.48, unit='m')) |
    (sc.abs(ds.coords['position'].fields.y - y_offset) > sc.scalar(0.45, unit='m')))

mask_center = sc.sqrt(
    ds.coords['position'].fields.x**2 +
    ds.coords['position'].fields.y**2) < sc.scalar(0.04, unit='m')
# cylindrical_radius = sc.sqrt(
#     ds.coords['position'].fields.x**2 +
#     ds.coords['position'].fields.y**2)
# mask_center = (cylindrical_radius < sc.scalar(0.04, unit='m')) | (cylindrical_radius > sc.scalar(0.4, unit='m'))

for key in ds:
    ds[key].masks['edges'] = mask_edges
    ds[key].masks['center'] = mask_center

In [ ]:
scn.instrument_view(ds['sample'], pixel_size=0.0075)

## Convert data to wavelength

In [ ]:
graph = sans.conversions.sans_elastic()
sc.show_graph(graph, simplified=True)

In [ ]:
from scipp.constants import g
ds.coords["gravity"] = sc.vector(value=[0, -1, 0]) * g
ds_wav = ds.transform_coords("wavelength", graph=graph)

# Bin the event data to the requested wavelength range
ds_wav = sc.Dataset(
    data={key: sc.bin(ds_wav[key],
                      edges=[sc.concat([wavelength_bins.min(), wavelength_bins.max()], dim='wavelength')])
          for key in ds_wav})

ds_wav

In [ ]:
graph_monitor = sans.conversions.sans_monitor()
for key in ds:
    for m in ['monitor2', 'monitor4']:
        ds_wav[key].attrs[m].value = ds[key].attrs[m].value.transform_coords("wavelength", graph=graph_monitor)

## Compute normalization term

### Direct beam function

In [ ]:
# Load direct beam function for main detector
direct_beam = loki.io.load_rkh_wav(loki.data.get_path('DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.dat'))

# Interpolate the direct beam function to the requested binning.
# WARNING: this removes the error bars on the direct beam function,
# because interpolating error bars is ill-defined.
from scipp.interpolate import interp1d
func = interp1d(sc.values(direct_beam), 'wavelength')
direct_beam = func(wavelength_bins, midpoints=True)

In [ ]:
sc.plot(direct_beam)

### Subtract background from monitors and align them to the same wavelength range

In [ ]:
threshold = 30.0 * sc.units.counts # Change to a wavelength range instead of a threshold

sample_incident_monitor = sans.normalization.subtract_background_and_rebin(
    ds_wav['sample'].attrs["monitor2"].value, wavelength_bins, threshold)
sample_transmission_monitor = sans.normalization.subtract_background_and_rebin(
    ds_wav['sample'].attrs["monitor4"].value, wavelength_bins, threshold)
direct_incident_monitor = sans.normalization.subtract_background_and_rebin(
    ds_wav['direct'].attrs["monitor2"].value, wavelength_bins, threshold)
direct_transmission_monitor = sans.normalization.subtract_background_and_rebin(
    ds_wav['direct'].attrs["monitor4"].value, wavelength_bins, threshold)

### Transmission fraction

In [ ]:
transmission_fraction = sans.normalization.transmission_fraction(
    data_incident_monitor=sample_incident_monitor,
    data_transmission_monitor=sample_transmission_monitor,
    direct_incident_monitor=direct_incident_monitor,
    direct_transmission_monitor=direct_transmission_monitor)
transmission_fraction

In [ ]:
transmission_fraction.plot()

### Solid Angle

In [ ]:
solid_angle = sans.normalization.solid_angle(ds['sample'],
                                             pixel_width=ds.coords['pixel_width'],
                                             pixel_height=ds.coords['pixel_height'])
solid_angle

### The denominator term

We keep the coordinate of the `direct_beam` term because it is bin centers,
which enables us to use `histogram` further down.
`sample_incident_monitor` and `transmission_fraction` both have bin-edge coordinates.

In [ ]:
denominator = solid_angle * direct_beam * sample_incident_monitor * transmission_fraction
denominator.coords['position'] = ds['sample'].coords['position']
denominator.coords['gravity'] = ds['sample'].coords['gravity']
denominator.coords['sample_position'] = ds['sample'].coords['sample_position']
denominator.coords['source_position'] = ds['sample'].coords['source_position']

from ess.wfm.tools import to_bin_centers
denominator.coords['wavelength'] = to_bin_centers(denominator.coords['wavelength'],
                                                  dim='wavelength')
denominator

In [ ]:
sc.plot(denominator.sum('spectrum'), norm='log')

## Convert to Q

In [ ]:
ds_q = ds_wav.transform_coords("Q", graph=graph)
ds_q

In [ ]:
q_edges = sc.array(dims=['Q'], values=[0.01, 0.6], unit='1/angstrom')
sample_q_binned = sc.bin(ds_q['sample'], edges=[q_edges])
sample_q_binned

In [ ]:
sample_q_summed = sample_q_binned.bins.concat('spectrum')
sample_q_summed

In [ ]:
sc.plot(sample_q_summed, norm="log")

### Convert denominator to Q

In [ ]:
denominator_q = denominator.transform_coords("Q", graph=graph)
denominator_q

In [ ]:
hist = sc.histogram(denominator_q,
                    bins=sc.linspace(dim='Q', start=0.01, stop=0.6, num=141, unit='1/angstrom'))
den_q_summed = hist.sum('spectrum')
den_q_summed.plot(norm='log')

## Normalize the sample

In [ ]:
sample_normalized = sample_q_summed.bins / sc.lookup(func=den_q_summed, dim='Q')
sample_normalized

In [ ]:
sc.plot(sample_normalized)